## Data Science - Group 15

### Topic: Toronto E-Bike demographic, travel and public opinion data

In [50]:
##### Data Ingestion

import pandas as pd

raw_data_path = "../data/E-Bike_Survey_Responses.csv"
columnNames = names=['TimeStamp','AgeRange','Sex','PhysicalHealth','Education','HouseholdIncome','EmploymentStatus','District','AvgDistanceTravel','AvgCommute','PreferedTransportation','PrivateMotorizedVehicles','SupportingStatements','MultiUseTrails','AwarePathSpeedLimit','WitnessedCollisionConflict','OpinionSpeedLimit','OpinionPathBylaw','BicycleLaneUsage','OpinionLaneBylaw','OpinionSidewalk','OpinionPersonalMobilityDevice'] # names of columns

survey = pd.read_csv(raw_data_path, names=columnNames, skiprows=1)

survey.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2238 entries, 0 to 2237
Data columns (total 22 columns):
TimeStamp                        2238 non-null object
AgeRange                         2234 non-null object
Sex                              2221 non-null object
PhysicalHealth                   2227 non-null object
Education                        2220 non-null object
HouseholdIncome                  2178 non-null object
EmploymentStatus                 2223 non-null object
District                         2237 non-null object
AvgDistanceTravel                2237 non-null object
AvgCommute                       2237 non-null object
PreferedTransportation           2238 non-null object
PrivateMotorizedVehicles         2238 non-null object
SupportingStatements             2238 non-null object
MultiUseTrails                   2238 non-null object
AwarePathSpeedLimit              2238 non-null object
WitnessedCollisionConflict       2238 non-null object
OpinionSpeedLimit            

In [51]:
survey = survey.dropna()
survey.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2138 entries, 0 to 2237
Data columns (total 22 columns):
TimeStamp                        2138 non-null object
AgeRange                         2138 non-null object
Sex                              2138 non-null object
PhysicalHealth                   2138 non-null object
Education                        2138 non-null object
HouseholdIncome                  2138 non-null object
EmploymentStatus                 2138 non-null object
District                         2138 non-null object
AvgDistanceTravel                2138 non-null object
AvgCommute                       2138 non-null object
PreferedTransportation           2138 non-null object
PrivateMotorizedVehicles         2138 non-null object
SupportingStatements             2138 non-null object
MultiUseTrails                   2138 non-null object
AwarePathSpeedLimit              2138 non-null object
WitnessedCollisionConflict       2138 non-null object
OpinionSpeedLimit            

In [52]:
#Drop TimeStamp Column, it does not provide any useful information
survey = survey.drop('TimeStamp', axis = 1)

In [53]:
#print(survey['Sex'].value_counts()) #Light Cleaning needed - some non-standard/open ended replies 
#print(survey['AgeRange'].value_counts()) # Is Clean
#print(survey['PhysicalHealth'].value_counts()) #Light Cleaning needed - some non-standard/open ended replies 
#print(survey['Education'].value_counts()) #Light Cleaning needed - some non-standard/open ended replies
#print(survey['HouseholdIncome'].value_counts()) #Is Clean
#print(survey['EmploymentStatus'].value_counts()) #Light Cleaning needed - some non-standard/open ended replies 
#print(survey['District'].value_counts()) #Heavy cleaning needed - alot of non-standard/open ended replies
#print(survey['AvgDistanceTravel'].value_counts())#Is Clean
#print(survey['AvgCommute'].value_counts())#Is Clean
#print(survey['PreferedTransportation'].value_counts())#Light Cleaning needed - some non-standard/open ended responces 
#print(survey['PrivateMotorizedVehicles'].value_counts())#Heavy cleaning needed - alot of non-standard/open ended responces
#print(survey['SupportingStatements'].value_counts()) #Moderate Cleaning needed - various combinations of responces, all appear to be bound
#print(survey['MultiUseTrails'].value_counts()) #Moderate Cleaning needed - various combinations of responces, all appear to be bound
#print(survey['AwarePathSpeedLimit'].value_counts())# Is Clean
#print(survey['WitnessedCollisionConflict'].value_counts())#Moderate Cleaning needed - various combinations of responces, all appear to be bound
#print(survey['OpinionSpeedLimit'].value_counts())#Heavy Cleaning needed - various combinations of responces and openeded responces
#print(survey['OpinionPathBylaw'].value_counts())  #Moderate Cleaning needed - various combinations of responces and openeded responces        
#print(survey['BicycleLaneUsage'].value_counts()) #Moderate Cleaning needed - various combinations of responces and openeded responces       
#print(survey['OpinionLaneBylaw'].value_counts()) #Heavy Cleaning needed - various combinations of responces and openeded responces
#print(survey['OpinionSidewalk'].value_counts()) #Moderate Cleaning needed - various combinations of responces and openeded responces           
#print(survey['OpinionPersonalMobilityDevice'].value_counts()) #Moderate Cleaning needed - various combinations of responces and openeded responces                     
               

#As I see it our options (or combination of options) to clean non-standard/open ended responces:
# 1) Drop rows
# 2) Group non-standard/open ended responces it's own category
# 3) Try to fit non-standard/open ended responces into existing categories

#my intial thought it #2 for majority features

#For the data that has various combinations, i.e. those that appear as if they were picked from a picker list, we need to identify all the unique options and: 
#1) do some sort of multi-index
#2) create new Categorical columns - Yes and No for each unique option

#my intial thought it #2 for majority of the feature



In [54]:
#Useful Function

#Function that retruns unique list from a given list
def unique_list(lst):
    unique_list = []
    for x in lst:
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

#Function that retruns datafame with all values other then 'keepValues' in 'feature' replaced with 'replaceWith' in Dataframe 'df'
def Replace_All_Other_Values(df, feature,keepValues,replaceWith):
    for item in df[feature]:
        if item in keepValues:
            pass
        else:
            df = df.replace({feature: item},replaceWith)
    return df


    

In [63]:
#Clean Sex feature
survey = Replace_All_Other_Values(survey, 'Sex',['Male','Female'], 'Other')
#survey['Sex'].value_counts()

#Clean PhysicalHealth feature
survey = Replace_All_Other_Values(survey, 'PhysicalHealth',['Good','Excellent','Very good','Fairly good','Poor'], 'Other')
#survey['PhysicalHealth'].value_counts()

#Clean Education feature
survey = Replace_All_Other_Values(survey, 'Education',['University degree','Post graduate','College or trade school diploma','High school diploma '], 'Other')
#survey['Education'].value_counts()



#Clean Education feature
survey = Replace_All_Other_Values(survey, 'EmploymentStatus',['Full Time','Self Employed','Retired','Part Time','Student','Unemployed','Home Maker'], 'Other')
#survey['EmploymentStatus'].value_counts()            

                                       
                      



Full Time        1358
Self Employed     344
Retired           127
Part Time         117
Student           114
Unemployed         35
Other              26
Home Maker         17
Name: EmploymentStatus, dtype: int64